<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px; text-align: center">
    <div style="padding-bottom: 100px">
        <img src="../images/pycon_title.svg">
    </div>
    <div style="font-weight: bold; font-size: 8.5em;">High Performance Pandas</div>
    <div style="font-size: 4.5em">Leveraging the power of pandas' data manipulation</div>
    <div style="font-size: 1.5em; font-style: italic; text-align: left; padding-left: 50px; text-align:center; padding-top: 20px">
        Feb 8-10, Bogot&aacute;, Colombia <br>
        Luis David Camacho<br>
        Lead Data Architect, Cyxtera Techonologies, Inc.
    </div>
</div>

<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
    <div style="font-weight: bold; font-size: 5em; padding-left: 50px">Outline:</div>
    <div style="display: inline-grid; width: 35%; text-align: justify; font-size: 1.15em; font-family: helvetica; padding-left: 100px;">
        <ul style="font-size:1.7em; list-style-image: url(../files/images/python_bullet_point.png)">
            <li>Intro</li>
            <li>When Not to Use pandas</li>
            <li>Combining DataFrames:
                <ul style="list-style-image: url(../files/images/python_bullet_point.png)">
                    <li>Merge</li>
                    <li>Concat</li>
                    <li>Append</li>
                </ul>
            </li>
            <li>Functional Approach:
                <ul style="list-style-image: url(../files/images/python_bullet_point.png)">
                    <li>Employing Lambda functions</li>
                    <li>Employing defined functions</li>
                </ul>
            </li>
            <li>Data Querying:
                <ul style="list-style-image: url(../files/images/python_bullet_point.png)">
                    <li>The .at &amp; .iat operator</li>
                    <li>Loc, query, eval</li>
                    <li>Multi-level index</li>
                </ul>
            </li>
            <li>Series and DataFrames Operations</li>
            <li>Selecting Columns</li>
            <li>Windowing &amp; Expanding</li>
            <li>Grouping and Aggregations:
                <ul style="list-style-image: url(../files/images/python_bullet_point.png)">
                    <li>.groupby</li>
                    <li>Merge, join, concat, &amp; append</li>
                    <li>Aggregations</li>
                </ul>
            </li>
<!--             <li>Best Practices:
                <ul style="list-style-image: url(../files/images/python_bullet_point.png)">
                    <li>Creating features data set</li>
                    <li>Building importable dictionaries</li>
                    <li>Memory management</li>
                </ul>
            </li> -->
            <!-- <li>Clustered pandas</li> -->
        </ul>
    </div>
    <div style="display: inline-grid; width: 55%;height: 500px; background-image:url(../files/images/pandas.jpg); background-size: 800px 450px; background-repeat: no-repeat;  background-position: center; float: right; padding-bottom: 80px; padding-top: 150px; padding-right: 50px"></div>
    <div style="widht:150px;heigth: 100px; position: absolute;bottom: 2%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>
</div>

<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 5em;">Intro</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-left: 50px; padding-right: 100px;">
    As software engineers and data scientists, we are constantly dealing with a big challenge: <span style="font-size: 1.3em;">data manipulation!</span> The proper tool to use for a given project usually depends on how much data you're dealing with, but when your goal is to generate insights based on data or to create a machine learning algorithm, what you want is a fast way to query and transform this data. Pandas is among the most powerful python tools that can help you to overcome that challenge, allowing you to store large amounts of data in memory with a powerful API that simplifies the data querying and transformation code with high performance.
</p>
<div style="text-align: justify; font-size: 1.8em; font-family: helvetica; font-style: italic; padding-bottom: 80px; padding-left: 100px; padding-right: 100px;">
    <span>Current Pandas API:</span>
    <ul style="list-style-image: url(../files/images/python_bullet_point.png)">
        <li>Series Object has 325 public methods/attributes</li>
        <li>Data Frame Object has 224 public methods/attributes</li>
        <li>Pandas has native support for 14 data formats and Python objects</li>
    </ul>
</div>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>
<!-- mention the size of public methods of API -->

In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import re
from markupsafe import escape
from IPython.display import display, Markdown
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

In [2]:
#constants
base_path = '../data/raw/'
html_path = base_path+'html/'

In [3]:
with open(base_path+'domains.list') as domains_file:
    domains = domains_file.readlines()

# show content sample (5 lines)
for dom in domains[:5]:
    display(Markdown('<span style="font-size: 2em">{}</span>'.format(dom)))

<span style="font-size: 2em">0123movies.com
</span>

<span style="font-size: 2em">0123movies.org
</span>

<span style="font-size: 2em">0123movies.sc
</span>

<span style="font-size: 2em">0123putlocker.com
</span>

<span style="font-size: 2em">037hd.com
</span>

In [4]:
# collect html data from domain
def gather_html(domain_url):
    with open(html_path+domain_url.strip()+'.html', 'r') as doc:
        content = doc.read()
    return content

<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 5em;">When Not to Use Pandas</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 50px; padding-right: 100px;">
Though pandas is a very powerful tool, as with any other tool in the market, <span style="font-size: 1.3em">it is not a silver bullet!</span> There are some cases in which pandas, no matter what, does not perform well. Let's look at some examples:
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px; ">Collecting Raw Data to Create a Start Data Set</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 150px; padding-right: 100px;">
As pandas is made up of a set of objects, its structure is very complex, meaning that creating a Series object or a DataFrame object is a heavy task. Creation does work quickly within pandas if you have a list of values. However, when you have a list of references, such as URLs, and what you need is the HTML data from these URLs, you must collect the information using dictionaries and, once you have the dictionary ready, you can build a DataFrame object from the dictionary. Avoid creating a void DataFrame object and add a Series of one register to it. 
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [5]:
%%time
start_df = pd.DataFrame()
for domain in domains[:10000]:
    start_df = start_df.append(pd.Series([gather_html(domain)], name=domain.strip()))

CPU times: user 17 s, sys: 1.39 s, total: 18.3 s
Wall time: 21.8 s


In [6]:
start_df.columns = ['content']
start_df.head()

,content
0123movies.com,&lt;!DOCTYPE html&gt;\n&lt;html xmlns=&#34;htt...
0123movies.org,&lt;!DOCTYPE html&gt;\n&lt;html lang=&#34;en&#...
0123movies.sc,&lt;!DOCTYPE html&gt;\n&lt;html lang=&#34;en&#...
0123putlocker.com,&lt;!DOCTYPE HTML&gt;\n&lt;html lang=&#34;en-U...
037hd.com,&lt;!DOCTYPE html PUBLIC &#34;-//W3C//DTD XHTM...


In [7]:
%%time
doms = dict()
for domain in domains[:10000]:
    doms[domain.strip()] = {"content": gather_html(domain)}
pandas_start_df = pd.DataFrame.from_dict(data=doms,orient='index')

CPU times: user 2.25 s, sys: 1.03 s, total: 3.28 s
Wall time: 3.28 s


In [8]:
pandas_start_df.head()

,content
0123movies.com,&lt;!DOCTYPE html&gt;\n&lt;html xmlns=&#34;htt...
0123movies.org,&lt;!DOCTYPE html&gt;\n&lt;html lang=&#34;en&#...
0123movies.sc,&lt;!DOCTYPE html&gt;\n&lt;html lang=&#34;en&#...
0123putlocker.com,&lt;!DOCTYPE HTML&gt;\n&lt;html lang=&#34;en-U...
037hd.com,&lt;!DOCTYPE html PUBLIC &#34;-//W3C//DTD XHTM...


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px">Avoid Looping with Pandas</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 150px; padding-right: 100px;">
Looping is a common task used to calculate specified metrics using previously collected data. This could be simply counting, filtering data, standardizing data types, etc. Pandas is made to help you with those kind of tasks, and it is very common to see code for those tasks made over the dictionaries before creating the data set. It is also easy to think: <span style="font-style:italic">'Yes!. There are methods for counting, for example, but pandas is slow to perform complex data transformations'</span>. In fact, pandas is powerful because its API allows it to apply user-defined transformations over one Series or multiple Series contained in the DataFrame.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [9]:
df_registers = len(start_df)
pandas_df_registers = len(pandas_start_df)

In [10]:
search_criteria = '(40\d (Forbidden|Not Found))'
error_domains = set()

In [11]:
%%time
#filtering 40x errors in HTML
for domain_row in start_df.iterrows():
    if re.search(search_criteria, domain_row[1].content, re.I):
        error_domains.add(domain_row[0])
start_df.drop(error_domains, inplace=True)

CPU times: user 20.1 s, sys: 0 ns, total: 20.1 s
Wall time: 20.1 s


In [13]:
print('total domains: '+ str(df_registers))
print('# of domains to drop: ' + str(len(error_domains)))
print('total domains: '+ str(len(start_df)))

total domains: 10000
# of domains to drop: 281
total domains: 9719


In [14]:
%%time
pandas_start_df = pandas_start_df.loc[(pandas_start_df.content.apply(lambda x: False if re.search(search_criteria, x, flags=re.I) else True))]

CPU times: user 18.8 s, sys: 0 ns, total: 18.8 s
Wall time: 18.8 s


In [16]:
print('total domains: '+ str(pandas_df_registers))
print('# of domains to drop: ' + str(pandas_df_registers - len(pandas_start_df)))
print('total domains: '+ str(len(pandas_start_df)))

total domains: 10000
# of domains to drop: 281
total domains: 9719


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 5em;">Combining DataFrames</div>
<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-left: 50px; padding-right: 100px;">
There are a variety of situations in which you have multiple DataFrames with complementary information. Three common cases are:
</p>
<ol style="font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 250px;">
    <li>Vertical merge: add rows from the second DataFrame to the first DataFrame.</li>
    <li>Horizontal merge: add columns from the second DataFrame to the first DataFrame.</li>
    <li>Selective merge: combine the information of two data sets based on defined conditions.</li>
</ol>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [20]:
# new data set with more rows
doms = dict()
for domain in domains[10000:20000]:
    doms[domain.strip()] = {"content": gather_html(domain)}
additional_start_df = pd.DataFrame.from_dict(data=doms,orient='index')

In [21]:
# new data set with more columns
with open('../data/raw/whois.list') as whois_file:
    whois_df = pd.DataFrame.from_dict(eval(whois_file.read()), orient='index')

In [22]:
# new data set that requires a join
with open('../data/raw/certs.list') as certs_file:
    certs_df = pd.DataFrame.from_dict(eval(certs_file.read()), orient='index', columns=['cert'])
    certs_df.cert = certs_df.cert.str.encode('utf-8')

<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px">Append</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 150px; padding-right: 100px;">
The append method adds rows to the end of a DataFrame object. This method is useful for combining similar objects and outputs a new, unified DataFrame object.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [23]:
%%time
# add rows to the data frame
full_df = pandas_start_df.append(additional_start_df)

CPU times: user 3.05 ms, sys: 0 ns, total: 3.05 ms
Wall time: 2.85 ms


In [24]:
print('starting shape: ' + str(pandas_start_df.shape))
print('new shape: ' + str(full_df.shape))
full_df.head()

starting shape: (9719, 1)
new shape: (19719, 1)


,content
0123movies.com,&lt;!DOCTYPE html&gt;\n&lt;html xmlns=&#34;htt...
0123movies.org,&lt;!DOCTYPE html&gt;\n&lt;html lang=&#34;en&#...
0123movies.sc,&lt;!DOCTYPE html&gt;\n&lt;html lang=&#34;en&#...
0123putlocker.com,&lt;!DOCTYPE HTML&gt;\n&lt;html lang=&#34;en-U...
037hd.com,&lt;!DOCTYPE html PUBLIC &#34;-//W3C//DTD XHTM...


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px">Concat</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 150px; padding-right: 100px;">
Concat method gives you the option of joining two or more DataFrames or Series objects in a defined axis (columns or rows). This method allows you to specify joint conditions, mirroring the functions of the merge method.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [25]:
%%time
full_whois = pd.concat([full_df, whois_df], sort=False)

CPU times: user 10.5 ms, sys: 0 ns, total: 10.5 ms
Wall time: 8.76 ms


In [26]:
print('starting shape: ' + str(full_df.shape))
print('new shape: ' + str(full_whois.shape))
full_whois.head()

starting shape: (19719, 1)
new shape: (27127, 7)


,content,QueryString,ServerName,ErrorCode,FoundMatch,WhoisRecord,QueryResult
0123movies.com,&lt;!DOCTYPE html&gt;\n&lt;html xmlns=&#34;htt...,NaN,NaN,NaN,NaN,NaN,NaN
0123movies.org,&lt;!DOCTYPE html&gt;\n&lt;html lang=&#34;en&#...,NaN,NaN,NaN,NaN,NaN,NaN
0123movies.sc,&lt;!DOCTYPE html&gt;\n&lt;html lang=&#34;en&#...,NaN,NaN,NaN,NaN,NaN,NaN
0123putlocker.com,&lt;!DOCTYPE HTML&gt;\n&lt;html lang=&#34;en-U...,NaN,NaN,NaN,NaN,NaN,NaN
037hd.com,&lt;!DOCTYPE html PUBLIC &#34;-//W3C//DTD XHTM...,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
full_whois.tail()

,content,QueryString,ServerName,ErrorCode,FoundMatch,WhoisRecord,QueryResult
zr.ru,NaN,zr.ru,whois.tcinet.ru,Success,Yes,"{'CreatedDate': '1997-08-05T18:16:02Z', 'Expir...",NaN
zuanke8.com,NaN,zuanke8.com,whois.dns.com.cn,Success,Yes,"{'CreatedDate': '2008-12-09T00:00:00Z', 'Updat...","{'QueryString': 'dom zuanke8.com', 'ServerName..."
zulily.com,NaN,zulily.com,whois.safenames.net,Success,Yes,"{'CreatedDate': '2009-12-10T02:16:31Z', 'Updat...","{'QueryString': 'dom zulily.com', 'ServerName'..."
zvooq.online,NaN,zvooq.online,whois.centralnic.com,Success,Yes,"{'CreatedDate': '2017-08-31T12:03:17Z', 'Updat...",NaN
zxxk.com,NaN,zxxk.com,whois.cnolnic.com,Success,Yes,"{'CreatedDate': '2003-12-23T00:00:00Z', 'Updat...","{'QueryString': 'dom zxxk.com', 'ServerName': ..."


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px">Merge</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 150px; padding-right: 100px;">
The merge method is the most customizable method to combine DataFrame elements. It allows you to join two DataFrames and, if needed, do so in multiple columns.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [28]:
%%time
# inner join using merge
result = pd.merge(full_df, whois_df, how='inner', left_index=True, right_index=True)

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 15.2 ms


In [29]:
result[['QueryString','ServerName','ErrorCode', 'FoundMatch']].head()

,QueryString,ServerName,ErrorCode,FoundMatch
0123movies.com,0123movies.com,whois.name.com,Success,Yes
037hd.com,037hd.com,whois.publicdomainregistry.com,Success,Yes
0daydown.com,0daydown.com,whois.godaddy.com,Success,Yes
10010.com,10010.com,whois.paycenter.com.cn,Success,Yes
1001fonts.com,1001fonts.com,whois.rrpproxy.net,Success,Yes


In [30]:
%%time
# left join using merge
outer = pd.merge(full_df, whois_df, how='outer', left_index=True, right_index=True)

CPU times: user 39.4 ms, sys: 292 µs, total: 39.7 ms
Wall time: 37.6 ms


In [31]:
outer[['QueryString','ServerName','ErrorCode', 'FoundMatch']].head()

,QueryString,ServerName,ErrorCode,FoundMatch
0123movies.com,0123movies.com,whois.name.com,Success,Yes
0123movies.org,NaN,NaN,NaN,NaN
0123movies.sc,NaN,NaN,NaN,NaN
0123putlocker.com,NaN,NaN,NaN,NaN
037hd.com,037hd.com,whois.publicdomainregistry.com,Success,Yes


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 5em;">Functional Approach</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 50px; padding-right: 100px;">
Pandas incorporates a functional approach that includes an apply method. This method allows for the execution of a user-defined or anonymous function <span style="font-style: italic">(know as lambda function in python)</span> on Series or DataFrame values. This is the most important functionality available to avoid looping or extracting data from DataFrames and then adding it the transformed result. The apply method will always output a series with the returned value of the passed function and the same index of the Series or DataFrame.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px">Employing Lambda Functions</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 150px; padding-right: 100px;">
The first step to using a lambda function is to pass a lambda definition as an argument of the apply method. The lambda argument allows only one value at a time from a Series; this means that if you're using DataFrame.apply(), the lambda definition will be executed once for each value in the Series and for each column in the DataFrame.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [32]:
%%time
# getting the certificate extension number
serial_numbers = list()
for cert in certs_df.cert.iteritems():
    serial_numbers.append(len(x509.load_pem_x509_certificate(cert[1], default_backend()).extensions))

CPU times: user 2.02 s, sys: 11.7 ms, total: 2.03 s
Wall time: 2.03 s


In [33]:
for sn in serial_numbers[:10]:
    print(sn)
print('trans len: ' + str(len(serial_numbers)))

10
10
9
9
9
9
10
9
10
10
trans len: 6139


In [34]:
%%time
# lambda function transformation
certs_df = certs_df.assign(extensions_number = certs_df.cert.apply(lambda cert: len(x509.load_pem_x509_certificate(cert, default_backend()).extensions)))

CPU times: user 2.23 s, sys: 0 ns, total: 2.23 s
Wall time: 2.23 s


In [35]:
certs_df.head()

,cert,extensions_number
037hd.com,b'-----BEGIN CERTIFICATE-----\nMIIGvTCCBmSgAwI...,10
0daydown.com,b'-----BEGIN CERTIFICATE-----\nMIIG6DCCBo6gAwI...,10
1001fonts.com,b'-----BEGIN CERTIFICATE-----\nMIIGRDCCBSygAwI...,9
1001freefonts.com,b'-----BEGIN CERTIFICATE-----\nMIIHJTCCBg2gAwI...,9
100noticias.com.ni,b'-----BEGIN CERTIFICATE-----\nMIIGJzCCBQ+gAwI...,9


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px">Employing User-Defined Functions</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 150px; padding-right: 100px;">
It is possible to pass a user-defined function to the apply method. However, it is important to be careful with the method definition as, when defining a lambda function, you can only receive one argument (in this case). When creating a user-defined method, a problem may occur if you receive multiple parameters.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [36]:
def parse_cert (cert_base64):
    cert_obj = x509.load_pem_x509_certificate(cert_base64, default_backend())
    return {'fingerprint': cert_obj.fingerprint(hashes.SHA1()).hex(),
           'issuer': ','.join([name.rfc4514_string() for name in cert_obj.issuer.rdns]),
           'subject': ','.join([name.rfc4514_string() for name in cert_obj.subject.rdns]),
           'not_valid_before': cert_obj.not_valid_before,
           'not_valid_after': cert_obj.not_valid_after}

In [37]:
%%time
certs_df.cert.apply(parse_cert).head()

CPU times: user 1.92 s, sys: 8 ms, total: 1.93 s
Wall time: 1.93 s


037hd.com             {'fingerprint': 'd73b2a77293784471b9b11cc794af...
0daydown.com          {'fingerprint': '7400f55dd495c12a02b4e702a136c...
1001fonts.com         {'fingerprint': 'e3800f6f28d1353d737c2aa00053e...
1001freefonts.com     {'fingerprint': '39f1e691b77ac75cbcd855c9ec741...
100noticias.com.ni    {'fingerprint': 'c3ad716b660050637920bcf65b92d...
Name: cert, dtype: object

In [38]:
json_normalize(certs_df.cert.apply(parse_cert)).head()

,fingerprint,issuer,not_valid_after,not_valid_before,subject
0,d73b2a77293784471b9b11cc794afae8b3b56372,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO ...",2019-03-21 23:59:59,2018-09-12 00:00:00,"OU=Domain Control Validated,OU=PositiveSSL Mul..."
1,7400f55dd495c12a02b4e702a136c33c8a18ffba,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO ...",2019-04-10 23:59:59,2018-10-02 00:00:00,"OU=Domain Control Validated,OU=PositiveSSL Mul..."
2,e3800f6f28d1353d737c2aa00053e655422e259f,"C=US,O=Let's Encrypt,CN=Let's Encrypt Authorit...",2018-11-21 03:29:16,2018-08-23 03:29:16,CN=1001fonts.net
3,39f1e691b77ac75cbcd855c9ec7411e13020bed0,"C=US,O=Let's Encrypt,CN=Let's Encrypt Authorit...",2018-12-28 05:55:10,2018-09-29 05:55:10,CN=1001freefonts.com
4,c3ad716b660050637920bcf65b92d5853da9b94b,"C=US,O=Let's Encrypt,CN=Let's Encrypt Authorit...",2018-11-10 04:29:44,2018-08-12 04:29:44,CN=100noticias.com.ni


In [39]:
full_certs_df = pd.merge(certs_df.reset_index(), json_normalize(certs_df.cert.apply(parse_cert)), left_index=True, right_index=True, how='inner').set_index(['index'])

In [40]:
full_certs_df.head()

,cert,extensions_number,fingerprint,issuer,not_valid_after,not_valid_before,subject
index,,,,,,,
037hd.com,b'-----BEGIN CERTIFICATE-----\nMIIGvTCCBmSgAwI...,10,d73b2a77293784471b9b11cc794afae8b3b56372,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO ...",2019-03-21 23:59:59,2018-09-12 00:00:00,"OU=Domain Control Validated,OU=PositiveSSL Mul..."
0daydown.com,b'-----BEGIN CERTIFICATE-----\nMIIG6DCCBo6gAwI...,10,7400f55dd495c12a02b4e702a136c33c8a18ffba,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO ...",2019-04-10 23:59:59,2018-10-02 00:00:00,"OU=Domain Control Validated,OU=PositiveSSL Mul..."
1001fonts.com,b'-----BEGIN CERTIFICATE-----\nMIIGRDCCBSygAwI...,9,e3800f6f28d1353d737c2aa00053e655422e259f,"C=US,O=Let's Encrypt,CN=Let's Encrypt Authorit...",2018-11-21 03:29:16,2018-08-23 03:29:16,CN=1001fonts.net
1001freefonts.com,b'-----BEGIN CERTIFICATE-----\nMIIHJTCCBg2gAwI...,9,39f1e691b77ac75cbcd855c9ec7411e13020bed0,"C=US,O=Let's Encrypt,CN=Let's Encrypt Authorit...",2018-12-28 05:55:10,2018-09-29 05:55:10,CN=1001freefonts.com
100noticias.com.ni,b'-----BEGIN CERTIFICATE-----\nMIIGJzCCBQ+gAwI...,9,c3ad716b660050637920bcf65b92d5853da9b94b,"C=US,O=Let's Encrypt,CN=Let's Encrypt Authorit...",2018-11-10 04:29:44,2018-08-12 04:29:44,CN=100noticias.com.ni


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 5em;">Data Querying</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 50px; padding-right: 100px;">
The very first step to leverage the power of pandas is to know how to use the most common methods of its API. These methods are built to be fast and simple, saving you the time it would take to code it by yourself.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [41]:
outer = outer.join(full_certs_df, how='outer')

<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px">The .at &amp; .iat Operator</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 150px; padding-right: 100px;">
Though it is advisable to avoid modifying individual values in a data frame, there may be some cases where it is necesary. If you find yourself in this situation, use the .at or .iat operators. The .at and .iat operators perform similarly to .loc and .iloc operators for querying, but are faster to assign values and allow you to modify an individual cell in place by default. Nonetheless, .at and .iat only work for modifying a single cell.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [42]:
%%time
for string in outer.QueryString.iteritems():
    if string[1] is np.nan:
        outer.at[string[0], 'QueryString'] = string[0]

CPU times: user 102 ms, sys: 3.69 ms, total: 105 ms
Wall time: 103 ms


In [43]:
outer[['QueryString']].head()

,QueryString
0123movies.com,0123movies.com
0123movies.org,0123movies.org
0123movies.sc,0123movies.sc
0123putlocker.com,0123putlocker.com
037hd.com,037hd.com


In [44]:
outer.columns

Index(['content', 'QueryString', 'ServerName', 'ErrorCode', 'FoundMatch',
       'WhoisRecord', 'QueryResult', 'cert', 'extensions_number',
       'fingerprint', 'issuer', 'not_valid_after', 'not_valid_before',
       'subject'],
      dtype='object')

In [45]:
%%time
cur_loc = 0
for string in outer.ErrorCode.iteritems():
    if string[1] is np.nan:
        outer.iat[cur_loc, 3] = 'Not Found'
    cur_loc += 1

CPU times: user 115 ms, sys: 3.63 ms, total: 119 ms
Wall time: 117 ms


In [46]:
outer[['ErrorCode']].head()

,ErrorCode
0123movies.com,Success
0123movies.org,Not Found
0123movies.sc,Not Found
0123putlocker.com,Not Found
037hd.com,Success


In [47]:
%%time
outer.extensions_number.fillna(0, inplace=True)

CPU times: user 0 ns, sys: 1.77 ms, total: 1.77 ms
Wall time: 1.07 ms


In [48]:
outer[['extensions_number']].head()

,extensions_number
0123movies.com,0.0
0123movies.org,0.0
0123movies.sc,0.0
0123putlocker.com,0.0
037hd.com,10.0


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 5em;">Multi-Level Indexes</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-left: 50px; padding-right: 100px;">
When you create a table in most database engines, it is mandatory to create an index. The DataFrame object in pandas is similar to a database table and is also mandatory to have and index. Indexing is critical as it creates a space to order the data contained in a table with one or more columns. This improves query perfomance over searching through randomly distributed data in the hard drive. These indexes must be carefully selected based on the data you most frequently filter.
</p>
<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 50px; padding-right: 100px;">
Pandas offers a multi-level index as a way to support multiple-column indexes. This feature is useful when you have multiple criteria to search for on your DataFrame. The multilevel index offers the same capabilities of a database multicolumn index.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [49]:
# new index level for data frame
outer.set_index('fingerprint', append=True, inplace=True, drop=False)

In [50]:
outer.head()

,,content,QueryString,ServerName,ErrorCode,FoundMatch,WhoisRecord,QueryResult,cert,extensions_number,fingerprint,issuer,not_valid_after,not_valid_before,subject
,fingerprint,,,,,,,,,,,,,,
0123movies.com,NaN,&lt;!DOCTYPE html&gt;\n&lt;html xmlns=&#34;htt...,0123movies.com,whois.name.com,Success,Yes,"{'CreatedDate': '2017-02-04T03:56:53Z', 'Updat...","{'QueryString': 'dom 0123movies.com', 'ServerN...",NaN,0.0,NaN,NaN,NaT,NaT,NaN
0123movies.org,NaN,&lt;!DOCTYPE html&gt;\n&lt;html lang=&#34;en&#...,0123movies.org,NaN,Not Found,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaT,NaT,NaN
0123movies.sc,NaN,&lt;!DOCTYPE html&gt;\n&lt;html lang=&#34;en&#...,0123movies.sc,NaN,Not Found,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaT,NaT,NaN
0123putlocker.com,NaN,&lt;!DOCTYPE HTML&gt;\n&lt;html lang=&#34;en-U...,0123putlocker.com,NaN,Not Found,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaT,NaT,NaN
037hd.com,d73b2a77293784471b9b11cc794afae8b3b56372,&lt;!DOCTYPE html PUBLIC &#34;-//W3C//DTD XHTM...,037hd.com,whois.publicdomainregistry.com,Success,Yes,"{'CreatedDate': '2015-05-22T14:10:04Z', 'Updat...","{'QueryString': 'dom 037hd.com', 'ServerName':...",b'-----BEGIN CERTIFICATE-----\nMIIGvTCCBmSgAwI...,10.0,d73b2a77293784471b9b11cc794afae8b3b56372,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO ...",2019-03-21 23:59:59,2018-09-12,"OU=Domain Control Validated,OU=PositiveSSL Mul..."


In [51]:
display(Markdown('<span style="font-size: 2em">{}</span>'.format(outer.columns)))

<span style="font-size: 2em">Index(['content', 'QueryString', 'ServerName', 'ErrorCode', 'FoundMatch',
       'WhoisRecord', 'QueryResult', 'cert', 'extensions_number',
       'fingerprint', 'issuer', 'not_valid_after', 'not_valid_before',
       'subject'],
      dtype='object')</span>

In [52]:
outer.set_index(['ServerName', 'extensions_number', 'issuer'], append=True, inplace=True, drop=False)

In [53]:
outer.head()

,,,,,content,QueryString,ServerName,ErrorCode,FoundMatch,WhoisRecord,QueryResult,cert,extensions_number,fingerprint,issuer,not_valid_after,not_valid_before,subject
,fingerprint,ServerName,extensions_number,issuer,,,,,,,,,,,,,,
0123movies.com,NaN,whois.name.com,0.0,NaN,&lt;!DOCTYPE html&gt;\n&lt;html xmlns=&#34;htt...,0123movies.com,whois.name.com,Success,Yes,"{'CreatedDate': '2017-02-04T03:56:53Z', 'Updat...","{'QueryString': 'dom 0123movies.com', 'ServerN...",NaN,0.0,NaN,NaN,NaT,NaT,NaN
0123movies.org,NaN,NaN,0.0,NaN,&lt;!DOCTYPE html&gt;\n&lt;html lang=&#34;en&#...,0123movies.org,NaN,Not Found,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaT,NaT,NaN
0123movies.sc,NaN,NaN,0.0,NaN,&lt;!DOCTYPE html&gt;\n&lt;html lang=&#34;en&#...,0123movies.sc,NaN,Not Found,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaT,NaT,NaN
0123putlocker.com,NaN,NaN,0.0,NaN,&lt;!DOCTYPE HTML&gt;\n&lt;html lang=&#34;en-U...,0123putlocker.com,NaN,Not Found,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaT,NaT,NaN
037hd.com,d73b2a77293784471b9b11cc794afae8b3b56372,whois.publicdomainregistry.com,10.0,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO CA Limited,CN=COMODO ECC Domain Validation Secure Server CA 2",&lt;!DOCTYPE html PUBLIC &#34;-//W3C//DTD XHTM...,037hd.com,whois.publicdomainregistry.com,Success,Yes,"{'CreatedDate': '2015-05-22T14:10:04Z', 'Updat...","{'QueryString': 'dom 037hd.com', 'ServerName':...",b'-----BEGIN CERTIFICATE-----\nMIIGvTCCBmSgAwI...,10.0,d73b2a77293784471b9b11cc794afae8b3b56372,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO ...",2019-03-21 23:59:59,2018-09-12,"OU=Domain Control Validated,OU=PositiveSSL Mul..."


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px">.loc, .query &amp; .eval</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 150px; padding-right: 100px;">
There is a time and a place for everything, and the same goes for querying data using pandas. However, each way has different performance and memory consumption. The following sections will compare the behavior on a small sample data set and describe how the process should behave in larger data sets. Nevertheless, all will obtain the same results.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [54]:
div_pattern = '({})'.format(escape('<div'))
script_pattern = '({})'.format(escape('<script'))
outer = outer.assign(div_count=outer.content.str.count(div_pattern),
            script_count=outer.content.str.count(script_pattern))

<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px">.loc Operator</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 150px; padding-right: 100px;">
The .loc operator performs well when querying data on indexes in small DataFrames. Compared to the .at operator, .loc is slower to set values and does not modify the original DataFrame; instead, it creates a copy. However, it exhibits similar querying perfomance. This method returns a subset of the DataFrame containing only the specified indexes; it also supports boolean indexing: a series of booleans with the same index as a DataFrame and only returns those with True.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [55]:
%%time
outer.loc['google.com']

CPU times: user 4.13 ms, sys: 8 µs, total: 4.13 ms
Wall time: 3.59 ms


,,,,content,QueryString,ServerName,ErrorCode,FoundMatch,WhoisRecord,QueryResult,cert,extensions_number,fingerprint,issuer,not_valid_after,not_valid_before,subject,div_count,script_count
fingerprint,ServerName,extensions_number,issuer,,,,,,,,,,,,,,,,
4c33f58ab299a41436e2423902240875e8cceacb,whois.markmonitor.com,8.0,"C=US,O=Google Trust Services,CN=Google Internet Authority G3",&lt;!doctype html&gt;&lt;html itemscope=&#34;&...,google.com,whois.markmonitor.com,Success,Yes,"{'CreatedDate': '1997-09-15T07:00:00Z', 'Updat...","{'QueryString': 'dom google.com', 'ServerName'...",b'-----BEGIN CERTIFICATE-----\nMIIIdDCCB1ygAwI...,8.0,4c33f58ab299a41436e2423902240875e8cceacb,"C=US,O=Google Trust Services,CN=Google Interne...",2018-11-20 18:23:00,2018-08-28 18:23:00,"C=US,ST=California,L=Mountain View,O=Google LL...",11.0,6.0


In [56]:
outer.loc[(slice(None), slice('4c33f58ab299a41436e2423902240875e8cceacb', '4c33f58ab299a41436e2423902240875e8cceacb')), :].head()

,,,,,content,QueryString,ServerName,ErrorCode,FoundMatch,WhoisRecord,QueryResult,cert,extensions_number,fingerprint,issuer,not_valid_after,not_valid_before,subject,div_count,script_count
,fingerprint,ServerName,extensions_number,issuer,,,,,,,,,,,,,,,,
ampproject.org,4c33f58ab299a41436e2423902240875e8cceacb,whois.publicinterestregistry.net,8.0,"C=US,O=Google Trust Services,CN=Google Internet Authority G3",&lt;!DOCTYPE html&gt;\n&lt;html ⚡ lang=&#34;en...,ampproject.org,whois.publicinterestregistry.net,ParseFailed,Yes,"{'CreatedDate': '2015-08-31T16:13:25Z', 'Updat...",NaN,b'-----BEGIN CERTIFICATE-----\nMIIIdDCCB1ygAwI...,8.0,4c33f58ab299a41436e2423902240875e8cceacb,"C=US,O=Google Trust Services,CN=Google Interne...",2018-11-20 18:23:00,2018-08-28 18:23:00,"C=US,ST=California,L=Mountain View,O=Google LL...",257.0,15.0
business.site,4c33f58ab299a41436e2423902240875e8cceacb,whois.centralnic.com,8.0,"C=US,O=Google Trust Services,CN=Google Internet Authority G3",&lt;!doctype html&gt;&lt;html lang=&#34;en&#34...,business.site,whois.centralnic.com,Success,Yes,"{'CreatedDate': '2017-04-17T08:53:39Z', 'Updat...",NaN,b'-----BEGIN CERTIFICATE-----\nMIIIdDCCB1ygAwI...,8.0,4c33f58ab299a41436e2423902240875e8cceacb,"C=US,O=Google Trust Services,CN=Google Interne...",2018-11-20 18:23:00,2018-08-28 18:23:00,"C=US,ST=California,L=Mountain View,O=Google LL...",3.0,8.0
doubleclick.net,4c33f58ab299a41436e2423902240875e8cceacb,whois.markmonitor.com,8.0,"C=US,O=Google Trust Services,CN=Google Internet Authority G3",&lt;!DOCTYPE html&gt;\n&lt;html class=&#34;glu...,doubleclick.net,whois.markmonitor.com,Success,Yes,"{'CreatedDate': '1996-01-16T05:00:00Z', 'Updat...","{'QueryString': 'dom doubleclick.net', 'Server...",b'-----BEGIN CERTIFICATE-----\nMIIIdDCCB1ygAwI...,8.0,4c33f58ab299a41436e2423902240875e8cceacb,"C=US,O=Google Trust Services,CN=Google Interne...",2018-11-20 18:23:00,2018-08-28 18:23:00,"C=US,ST=California,L=Mountain View,O=Google LL...",165.0,10.0
goo.gl,4c33f58ab299a41436e2423902240875e8cceacb,whois.nic.gl,8.0,"C=US,O=Google Trust Services,CN=Google Internet Authority G3",&lt;!DOCTYPE html&gt;&lt;html lang=en&gt;&lt;m...,goo.gl,whois.nic.gl,Success,Yes,"{'CreatedDate': '2005-06-22T02:00:00Z', 'Updat...",NaN,b'-----BEGIN CERTIFICATE-----\nMIIIdDCCB1ygAwI...,8.0,4c33f58ab299a41436e2423902240875e8cceacb,"C=US,O=Google Trust Services,CN=Google Interne...",2018-11-20 18:23:00,2018-08-28 18:23:00,"C=US,ST=California,L=Mountain View,O=Google LL...",1.0,0.0
google.com,4c33f58ab299a41436e2423902240875e8cceacb,whois.markmonitor.com,8.0,"C=US,O=Google Trust Services,CN=Google Internet Authority G3",&lt;!doctype html&gt;&lt;html itemscope=&#34;&...,google.com,whois.markmonitor.com,Success,Yes,"{'CreatedDate': '1997-09-15T07:00:00Z', 'Updat...","{'QueryString': 'dom google.com', 'ServerName'...",b'-----BEGIN CERTIFICATE-----\nMIIIdDCCB1ygAwI...,8.0,4c33f58ab299a41436e2423902240875e8cceacb,"C=US,O=Google Trust Services,CN=Google Interne...",2018-11-20 18:23:00,2018-08-28 18:23:00,"C=US,ST=California,L=Mountain View,O=Google LL...",11.0,6.0


In [57]:
outer.loc[(slice(None), slice(None), slice(None), slice(10,10)), :].head()

,,,,,content,QueryString,ServerName,ErrorCode,FoundMatch,WhoisRecord,QueryResult,cert,extensions_number,fingerprint,issuer,not_valid_after,not_valid_before,subject,div_count,script_count
,fingerprint,ServerName,extensions_number,issuer,,,,,,,,,,,,,,,,
037hd.com,d73b2a77293784471b9b11cc794afae8b3b56372,whois.publicdomainregistry.com,10.0,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO CA Limited,CN=COMODO ECC Domain Validation Secure Server CA 2",&lt;!DOCTYPE html PUBLIC &#34;-//W3C//DTD XHTM...,037hd.com,whois.publicdomainregistry.com,Success,Yes,"{'CreatedDate': '2015-05-22T14:10:04Z', 'Updat...","{'QueryString': 'dom 037hd.com', 'ServerName':...",b'-----BEGIN CERTIFICATE-----\nMIIGvTCCBmSgAwI...,10.0,d73b2a77293784471b9b11cc794afae8b3b56372,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO ...",2019-03-21 23:59:59,2018-09-12 00:00:00,"OU=Domain Control Validated,OU=PositiveSSL Mul...",184.0,16.0
0daydown.com,7400f55dd495c12a02b4e702a136c33c8a18ffba,whois.godaddy.com,10.0,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO CA Limited,CN=COMODO ECC Domain Validation Secure Server CA 2",﻿&lt;!DOCTYPE HTML&gt;\n&lt;html&gt;\n&lt;head...,0daydown.com,whois.godaddy.com,Success,Yes,"{'CreatedDate': '2013-04-28T14:17:43Z', 'Updat...","{'QueryString': 'dom 0daydown.com', 'ServerNam...",b'-----BEGIN CERTIFICATE-----\nMIIG6DCCBo6gAwI...,10.0,7400f55dd495c12a02b4e702a136c33c8a18ffba,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO ...",2019-04-10 23:59:59,2018-10-02 00:00:00,"OU=Domain Control Validated,OU=PositiveSSL Mul...",29.0,9.0
101.ru,046316795324401e6aa535d677f85c8d02dabb0f,whois.tcinet.ru,10.0,"C=US,O=DigiCert Inc,OU=www.digicert.com,CN=RapidSSL RSA CA 2018",NaN,101.ru,whois.tcinet.ru,Success,Yes,"{'CreatedDate': '1997-09-10T15:00:55Z', 'Expir...",NaN,b'-----BEGIN CERTIFICATE-----\nMIIFpDCCBIygAwI...,10.0,046316795324401e6aa535d677f85c8d02dabb0f,"C=US,O=DigiCert Inc,OU=www.digicert.com,CN=Rap...",2019-07-06 12:00:00,2018-06-06 00:00:00,CN=*.101.ru,NaN,NaN
10fastfingers.com,b749eef48bdd9808de2a0cb65cbd855dbc076286,whois.registrygate.com,10.0,"C=US,O=Amazon,OU=Server CA 1B,CN=Amazon",&lt;!DOCTYPE html&gt;\n&lt;html lang=&#34;en&#...,10fastfingers.com,whois.registrygate.com,Success,Yes,"{'CreatedDate': '2007-11-13T10:17:24Z', 'Updat...","{'QueryString': 'dom 10fastfingers.com', 'Serv...",b'-----BEGIN CERTIFICATE-----\nMIIFgTCCBGmgAwI...,10.0,b749eef48bdd9808de2a0cb65cbd855dbc076286,"C=US,O=Amazon,OU=Server CA 1B,CN=Amazon",2019-10-25 12:00:00,2018-09-25 00:00:00,CN=10fastfingers.com,26.0,7.0
10jqka.com.cn,6f89ea6c39cde66291f74b631b0ce7c9b508c94a,whois.cnnic.net.cn,10.0,"C=US,O=Entrust\, Inc.,OU=See www.entrust.net/legal-terms,OU=(c) 2012 Entrust\, Inc. - for authorized use only,CN=Entrust Certification Authority - L1K",&lt;!DOCTYPE html&gt;\n&lt;html&gt;\n&lt;head&...,10jqka.com.cn,whois.cnnic.net.cn,Success,Yes,"{'CreatedDate': '2003-08-26T23:03:05Z', 'Expir...",NaN,b'-----BEGIN CERTIFICATE-----\nMIIHSDCCBjCgAwI...,10.0,6f89ea6c39cde66291f74b631b0ce7c9b508c94a,"C=US,O=Entrust\, Inc.,OU=See www.entrust.net/l...",2018-10-24 09:08:04,2017-10-23 08:38:06,"C=CN,ST=Zhejiang,L=Hangzhou,O=Hithink RoyalFlu...",418.0,39.0


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px">.query Method</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-left: 150px; padding-right: 100px;">
The .query method, from the DataFrame object, receives a string that represents an expression and evaluates it. The expression syntax supported by this method could be plain python or numpy expression <span style="font-style: italic">(numexpr)</span>. This method also supports symbol and natural logical operators: and = &amp;, or = | and not = ~, and also supports other common arithmetic operators like &lt;,&gt;,==,!=,+,-,=, among others.
</p>
<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-left: 150px; padding-right: 100px;">
This method is faster compared to .loc query when your DataFrame contains more than 100,000 rows. But, it is also a very useful way to query data when you have a MultilevelIndex. One of the advantages of this method is that it uses cleaner syntax than the index query, and the query meta-syntax also allows you to use the DataFrame column names as variables and to use local user-defined variables (using the @ symbol to access them).
</p>
<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-left: 150px; padding-right: 100px;">
<span style="font-style: bold; font-size: 1.3em">Fun fact:</span> Using numexp with the query method is faster than using plain python when you have more than 200,000 rows.
</p>
<div style="display: block; background-image:url(../files/images/query-perf.png); background-size: cover; background-repeat: no-repeat;  background-position: center; padding-bottom: 80px; width: 700px; height: 450px; margin-left: 10%"></div>
<p style="text-align: justify; font-size: 1.2em;  font-style: italic; font-family: helvetica; padding-bottom: 80px; padding-left: 10%; padding-right: 100px;">
    Query method comparison using python and numexpr syntax<br>
    taken from: <a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#performance-of-query">https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#performance-of-query</a>
</p>

<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 2%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [58]:
search_pattern = '\.com'

In [59]:
%%time
temp_df = outer.query('div_count > 100 and script_count > 1')

CPU times: user 29 ms, sys: 8.08 ms, total: 37 ms
Wall time: 34.3 ms


In [60]:
%%time
temp_df = outer.loc[(outer.div_count > 100) & (outer.script_count > 1)]

CPU times: user 6.29 ms, sys: 158 µs, total: 6.44 ms
Wall time: 5.39 ms


In [61]:
%%time
temp_df = outer.loc[(slice(None), slice(None), slice(None), slice(10,10)), :]

CPU times: user 13.5 ms, sys: 78 µs, total: 13.6 ms
Wall time: 10.9 ms


In [62]:
%%time
temp_df = outer.query('extensions_number == 10')

CPU times: user 6.65 ms, sys: 3.64 ms, total: 10.3 ms
Wall time: 8.75 ms


In [63]:
search_pattern = '\.com'

In [64]:
%%time
temp_df = outer.query('ilevel_0.str.contains(@search_pattern) and extensions_number == 10 and div_count > 100 and script_count > 1')

CPU times: user 27.4 ms, sys: 0 ns, total: 27.4 ms
Wall time: 24.9 ms


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px">.eval Method</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica;padding-left: 150px; padding-right: 100px;">
The .eval method is similar to the query method in large data sets and slower in small ones. It belongs to the pandas package and not the DataFrame object, meaning you need to specify the data frame you want to compare. It uses plain python syntax and it doesn't restrict to pandas objects, making it vulnerable to code injections. This method has similar performance to the .query method both in CPU usage and memory consumption. Nevertheless, both .query and .eval are able to use all the data frame and series methods.
</p>
<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 150px; padding-right: 100px;">
It is important to remark that the .eval method returns a Series object with boolean values and indexes, meaning you have to use index query based on the result. If you want to return a DataFrame from the .eval method, you have to write the whole expression; the .query method returns an actual DataFrame.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [65]:
%%time
temp_df = pd.eval('outer[outer.div_count > 100 and outer.script_count > 1]')

CPU times: user 23.1 ms, sys: 0 ns, total: 23.1 ms
Wall time: 27.2 ms


In [66]:
# eval method can evaluate any python expression
pd.eval('2**2')

4

<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 5em;">Series and DataFrames Operations</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-left: 50px; padding-right: 100px;">
Pandas offers not only Series and DataFrame objects, but a whole set of common statistical methods, plot methods, and overloaded operators to perform vectorized operations on Series objects.
</p>
<div style="display: inline-table; width: 100%; padding-left: 100px; padding-bottom: 80px">
    <div style="display: inline-row; width: 100%;">
        <div style="display: inline-grid; padding-left: 3em; width:20%">
            <span style="font-weight: bold; font-size: 2em">Operator Overloading</span>
        </div>
        <div style="display: inline-grid; font-size: 1.7em; max-width:60%; padding-left: 30px">
            Series and DataFrame objects overload the most common operator, allowing you to use simple syntax similar to the variables used in pure python. The overloaded operators are: <span style="font-weight: bold">=,==,+,-,*,/,**,%,&amp;,|,~,!=,&gt;,&lt;,&gt;=,&lt;=</span>.
        </div>
    </div>
    <div style="display: inline-row">
        <div style="display: inline-grid; padding-left: 3em; width:20%">
            <span style="font-weight: bold; font-size: 2em">.plot</span>
        </div>
        <div style="display: inline-grid; font-size: 1.7em; max-width:60%; padding-left: 30px">
        Pandas DataFrame includes a set of built-in plots. This prevents you from having to use external libraries for plotting and allows you to use the DataFrame column names as inputs. The implemented plot types are: 
            <ul style="list-style-image: url(../files/images/python_bullet_point.png)">
                <li>‘line’ : line plot (default)</li>
                <li>‘bar’ : vertical bar plot</li>
                <li>‘barh’ : horizontal bar plot</li>
                <li>‘hist’ : histogram</li>
                <li>‘box’ : boxplot</li>
                <li>‘kde’ : Kernel Density Estimation plot</li>
                <li>‘density’ : same as ‘kde’</li>
                <li>‘area’ : area plot</li>
                <li>‘pie’ : pie plot</li>
                <li>‘scatter’ : scatter plot</li>
                <li>‘hexbin’ : hexbin plot </li>
            </ul>
        </div>
    </div>
    <div style="display: inline-row">
        <div style="display: inline-grid; padding-left: 3em; width:20%">
            <span style="font-weight: bold; font-size: 2em">.to_latex</span>
        </div>
        <div style="display: inline-grid; font-size: 1.7em; max-width:60%; padding-left: 30px">
        Use this method to create a DataFrame or Series object into tabular latext. This requires the use of a package{booktabs}.
        </div>
    </div>
    <div style="display: inline-row">
        <div style="display: inline-grid; padding-left: 3em; width:20%">
            <span style="font-weight: bold; font-size: 2em">.json_normalize</span>
        </div>
        <div style="display: inline-grid; font-size: 1.7em; max-width:60%; padding-left: 30px">
        With a JSON data set, it is common to have a nested JSON in your list of objects. Using the pandas.from_record method turns these nested values into string columns; the pandas.io.json.json_normalize method automatically turns the JSON object into a flattened version with each of the fields creating a new column.
        </div>
    </div>
    <div style="display: inline-row">
        <div style="display: inline-grid; padding-left: 3em; width:20%">
            <span style="font-weight: bold; font-size: 2em">.to_dict</span>
        </div>
        <div style="display: inline-grid; font-size: 1.7em; max-width:60%; padding-left: 30px">
        This method turns a DataFrame or Series object into a python dictionary and allows you to specify the structure of the dictionary.
        </div>
    </div>
    <div style="display: inline-row">
        <div style="display: inline-grid; padding-left: 3em; width:20%">
            <span style="font-weight: bold; font-size: 2em">.from_dict</span>
        </div>
        <div style="display: inline-grid; font-size: 1.7em; max-width:60%; padding-left: 30px">
            This method reads the information from a python dictionary and turns it into a DataFrame. The dictionary must have one of the accepted dictionary structures.
        </div>
    </div>
    <div style="display: inline-row">
        <div style="display: inline-grid; padding-left: 3em; width:20%">
            <span style="font-weight: bold; font-size: 2em">.read_csv</span>
        </div>
        <div style="display: inline-grid; font-size: 1.7em; max-width:60%; padding-left: 30px">
            This method reads the information from a csv file. This file can be located in local storage or on the internet, in which case you can specify a URL. It also allows you to read compressed csv files by specifying the compression type, excluding zip format.
        </div>
    </div>
    <div style="display: inline-row">
        <div style="display: inline-grid; padding-left: 3em; width:20%">
            <span style="font-weight: bold; font-size: 2em">.describe</span>
        </div>
        <div style="display: inline-grid; font-size: 1.7em; max-width:60%; padding-left: 30px">
            This method generates basic statistics like median tendency, standard deviation, count of present values (not nan), and others.
        </div>
    </div>
    <div style="display: inline-row">
        <div style="display: inline-grid; padding-left: 3em; width:20%">
            <span style="font-weight: bold; font-size: 2em">.value_counts</span>
        </div>
        <div style="display: inline-grid; font-size: 1.7em; max-width:60%; padding-left: 30px">
            This method counts the present values in a Series object and records how many of each value are present. It returns a Series object with the unique values of the original Series as the index and the frequency of each individual value in the set.
        </div>
    </div>
    <div style="display: inline-row">
        <div style="display: inline-grid; padding-left: 3em; width:20%">
            <span style="font-weight: bold; font-size: 2em">.unique</span>
        </div>
        <div style="display: inline-grid; font-size: 1.7em; max-width:60%; padding-left: 30px">
            This method returns a list of different values, omitting the duplicates.
        </div>
    </div>
    <div style="display: inline-row">
        <div style="display: inline-grid; padding-left: 3em; width:20%">
            <span style="font-weight: bold; font-size: 2em">Series arithmetic functions</span>
        </div>
        <div style="display: inline-grid; font-size: 1.7em; max-width:60%; padding-left: 30px">
            There is a set of arithmetic methods that belong to a series. These methods are useful when you are performing a data analysis over the data or you want to find meaningful information over a data set. Some of these funcions are:
            <ul style="list-style-image: url(../files/images/python_bullet_point.png)">
                <li>.var: Calculates unbiased variance of a column or a row</li>
                <li>.std: Calculates the standard deviation of a column or a row</li>
                <li>.mean: Calculates the mean of a column or a row</li>
                <li>.median: Calculates the median tendency of a column or a row</li>
                <li>.quantile: Returns the values of a given quantile of a column or a row</li>
                <li>.count: Returns a Series of present values of a column or a row</li>
                <li>.sum: Returns the sum of the values present in a column or a row</li>
                <li>.min: Returns the minimum numeric value in a column or a row</li>
                <li>.max: Returns the maximum numeric value in a column or a row</li>
                <li>.abs: Returns a Series or DataFrame object with the absolute value of each numeric value. If the column has non-numeric values (except for Nan) it will fail; nonetheless, it supports timedelta objects</li>
                <li>.cumsum: Returns the cumulative sum of all elements in a Series</li>
            </ul>
        </div>
    </div>
</div>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 1%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 5em;">Selecting Columns</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 50px; padding-right: 100px;">
In the case that it is necessary to check some of the columns in a DataFrame, the most common way to do this is by iterating over the columns list of the DataFrame. To do this, search for a prefix in the name of each column or select the columns by type. Fortunately, pandas offers a simple way to select columns by type or name and returns a DataFrame only with the matching columns.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [67]:
# select columns by type
outer.select_dtypes('datetime64').head()

,,,,,not_valid_after,not_valid_before
,fingerprint,ServerName,extensions_number,issuer,,
0123movies.com,NaN,whois.name.com,0.0,NaN,NaT,NaT
0123movies.org,NaN,NaN,0.0,NaN,NaT,NaT
0123movies.sc,NaN,NaN,0.0,NaN,NaT,NaT
0123putlocker.com,NaN,NaN,0.0,NaN,NaT,NaT
037hd.com,d73b2a77293784471b9b11cc794afae8b3b56372,whois.publicdomainregistry.com,10.0,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO CA Limited,CN=COMODO ECC Domain Validation Secure Server CA 2",2019-03-21 23:59:59,2018-09-12


In [68]:
# code select columns by name (using .filter)
outer.filter(regex='_count$').head()

,,,,,div_count,script_count
,fingerprint,ServerName,extensions_number,issuer,,
0123movies.com,NaN,whois.name.com,0.0,NaN,105.0,21.0
0123movies.org,NaN,NaN,0.0,NaN,12.0,7.0
0123movies.sc,NaN,NaN,0.0,NaN,12.0,7.0
0123putlocker.com,NaN,NaN,0.0,NaN,8.0,1.0
037hd.com,d73b2a77293784471b9b11cc794afae8b3b56372,whois.publicdomainregistry.com,10.0,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO CA Limited,CN=COMODO ECC Domain Validation Secure Server CA 2",184.0,16.0


In [69]:
# code columns difference
outer.select_dtypes(exclude='object').head()

,,,,,extensions_number,not_valid_after,not_valid_before,div_count,script_count
,fingerprint,ServerName,extensions_number,issuer,,,,,
0123movies.com,NaN,whois.name.com,0.0,NaN,0.0,NaT,NaT,105.0,21.0
0123movies.org,NaN,NaN,0.0,NaN,0.0,NaT,NaT,12.0,7.0
0123movies.sc,NaN,NaN,0.0,NaN,0.0,NaT,NaT,12.0,7.0
0123putlocker.com,NaN,NaN,0.0,NaN,0.0,NaT,NaT,8.0,1.0
037hd.com,d73b2a77293784471b9b11cc794afae8b3b56372,whois.publicdomainregistry.com,10.0,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO CA Limited,CN=COMODO ECC Domain Validation Secure Server CA 2",10.0,2019-03-21 23:59:59,2018-09-12,184.0,16.0


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 5em;">Windowing &amp; Expanding</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-left: 50px; padding-right: 100px;">
A common task in data analysis is slicing data based on a fixed number of registers or a fixed period of time. To do that, the most common method is to create a loop to query the segment of data and then perform calculations. Nevertheless, pandas offers rolling and expanding methods to simplify this task.
</p>
<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-left: 50px; padding-right: 100px;">
The .rolling method, allows you to create a slice of data based on the specified number of rows. Additionaly, thanks to the big date types API, you can specify a time period for your window if you have a date type column or index.
</p>
<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-left: 50px; padding-right: 100px;">
The .expand method allows you to create a stating slice of data based on the specified number of rows and increment it to the end of the DataFrame.
</p>
<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 50px; padding-right: 100px;">
Both methods return a window object that is able to perform operations over the window in each iteration without looping.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [70]:
# rolling window
outer[['div_count', 'script_count']].rolling(10, min_periods=1).sum().head(15)

,,,,,div_count,script_count
,fingerprint,ServerName,extensions_number,issuer,,
0123movies.com,NaN,whois.name.com,0.0,NaN,105.0,21.0
0123movies.org,NaN,NaN,0.0,NaN,117.0,28.0
0123movies.sc,NaN,NaN,0.0,NaN,129.0,35.0
0123putlocker.com,NaN,NaN,0.0,NaN,137.0,36.0
037hd.com,d73b2a77293784471b9b11cc794afae8b3b56372,whois.publicdomainregistry.com,10.0,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO CA Limited,CN=COMODO ECC Domain Validation Secure Server CA 2",321.0,52.0
0933.me,NaN,NaN,0.0,NaN,375.0,56.0
0daydown.com,7400f55dd495c12a02b4e702a136c33c8a18ffba,whois.godaddy.com,10.0,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO CA Limited,CN=COMODO ECC Domain Validation Secure Server CA 2",404.0,65.0
0wikipedia.org,NaN,NaN,0.0,NaN,404.0,67.0
1-tube.ru,NaN,NaN,0.0,NaN,427.0,78.0


In [71]:
# code expanding window
outer[['div_count', 'script_count']].expanding(1).sum().head()

,,,,,div_count,script_count
,fingerprint,ServerName,extensions_number,issuer,,
0123movies.com,NaN,whois.name.com,0.0,NaN,105.0,21.0
0123movies.org,NaN,NaN,0.0,NaN,117.0,28.0
0123movies.sc,NaN,NaN,0.0,NaN,129.0,35.0
0123putlocker.com,NaN,NaN,0.0,NaN,137.0,36.0
037hd.com,d73b2a77293784471b9b11cc794afae8b3b56372,whois.publicdomainregistry.com,10.0,"C=GB,ST=Greater Manchester,L=Salford,O=COMODO CA Limited,CN=COMODO ECC Domain Validation Secure Server CA 2",321.0,52.0


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 5em;">Grouping &amp; Aggregations</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 50px; padding-right: 100px;">
There are situations in which you want to segregate information based on a category or multiple categories and then, on each segregated segment of information, you want to perform some calculations. Both tasks are grouping and aggregating information; again, pandas API is ready for it! The .groupby method allows you to choose columns to create groups and will return a group object. This object allows you to perform aggregation operations that return a new DataFrame object with the grouping columns as the index (one or more) in addition to the result of the aggregation.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px">.groupby</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 150px; padding-right: 100px;">
The group object has several built-in methods for aggregations. Most of these methods are valid for numerical series, but if you want to create a custom aggregation not implemented in pandas, you can also use the .apply method on groups. This will perform the apply operation for each group. It is important to note that grouping criteria can also be a lambda or custom defined function.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [72]:
outer.columns

Index(['content', 'QueryString', 'ServerName', 'ErrorCode', 'FoundMatch',
       'WhoisRecord', 'QueryResult', 'cert', 'extensions_number',
       'fingerprint', 'issuer', 'not_valid_after', 'not_valid_before',
       'subject', 'div_count', 'script_count'],
      dtype='object')

In [73]:
outer.index.names

FrozenList([None, 'fingerprint', 'ServerName', 'extensions_number', 'issuer'])

In [74]:
group = outer.groupby(by=['issuer','extensions_number'])

/home/knight/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'issuer' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.
/home/knight/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'extensions_number' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


In [75]:
print(type(group))
print(len(group.groups))

<class 'pandas.core.groupby.groupby.DataFrameGroupBy'>
309


<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 4em; padding-left: 100px">Aggregations</div>

<p style="text-align: justify; font-size: 2em; font-family: helvetica; padding-bottom: 80px; padding-left: 150px; padding-right: 100px;">
Aggregations can be calculated one-by-one or multiples at the same time, it is even possible to calculate multiple aggregations to one multiple columns selectively, all with just one line of code.
</p>
<!-- <div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>     -->
</div>

In [76]:
section_pattern = '({})'.format(escape('<section'))
canvas_pattern = '({})'.format(escape('<canvas'))
article_pattern = '({})'.format(escape('<article'))
nav_pattern = '({})'.format(escape('<nav'))
outer = outer.assign(section_count=outer.content.str.count(section_pattern),
            canvas_count=outer.content.str.count(canvas_pattern),
            article_count=outer.content.str.count(article_pattern),
            nav_count=outer.content.str.count(nav_pattern))

In [77]:
# sum aggregation
group[['ServerName']].count()

,,ServerName
issuer,extensions_number,
"1.2.840.113549.1.9.1=ssl@cnsrv.softlayer.com,CN=cnsrv.softlayer.com",3.0,3
"1.2.840.113549.1.9.1=ssl@single-4786.banahosting.com,CN=single-4786.banahosting.com",3.0,2
"C=--,ST=Neverhood,O=Something went wrong,CN=WRONG.hostname,1.2.840.113549.1.9.1=you@missed",3.0,9
"C=--,ST=SomeState,L=SomeCity,O=SomeOrganization,OU=SomeOrganizationalUnit,CN=EIG11,1.2.840.113549.1.9.1=root@EIG11",2.0,1
"C=--,ST=SomeState,L=SomeCity,O=SomeOrganization,OU=SomeOrganizationalUnit,CN=PA-Archives,1.2.840.113549.1.9.1=root@PA-Archives",2.0,1
"C=--,ST=SomeState,L=SomeCity,O=SomeOrganization,OU=SomeOrganizationalUnit,CN=be,1.2.840.113549.1.9.1=root@be",3.0,1
"C=--,ST=SomeState,L=SomeCity,O=SomeOrganization,OU=SomeOrganizationalUnit,CN=engin.engineersgarage.com,1.2.840.113549.1.9.1=root@engin.engineersgarage.com",2.0,1
"C=--,ST=SomeState,L=SomeCity,O=SomeOrganization,OU=SomeOrganizationalUnit,CN=iZ6wegvue12moarnsa2npaZ,1.2.840.113549.1.9.1=root@iZ6wegvue12moarnsa2npaZ",2.0,1
"C=--,ST=SomeState,L=SomeCity,O=SomeOrganization,OU=SomeOrganizationalUnit,CN=ip-10-0-3-103.us-west-2.compute.internal,1.2.840.113549.1.9.1=root@ip-10-0-3-103.us-west-2.compute.internal",2.0,1


In [78]:
# sum and count aggregation
outer.groupby(by=['issuer']).agg({'ServerName': ['count'], 'extensions_number': ['mean','max','min']}).head()

/home/knight/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: 'issuer' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  


ServerName  \
                                                        count   
issuer                                                          
1.2.840.113549.1.9.1=ssl@cnsrv.softlayer.com,CN...          3   
1.2.840.113549.1.9.1=ssl@single-4786.banahostin...          2   
C=--,ST=Neverhood,O=Something went wrong,CN=WRO...          9   
C=--,ST=SomeState,L=SomeCity,O=SomeOrganization...          1   
C=--,ST=SomeState,L=SomeCity,O=SomeOrganization...          1   

                                                   extensions_number            
                                                                mean  max  min  
issuer                                                                          
1.2.840.113549.1.9.1=ssl@cnsrv.softlayer.com,CN...               3.0  3.0  3.0  
1.2.840.113549.1.9.1=ssl@single-4786.banahostin...               3.0  3.0  3.0  
C=--,ST=Neverhood,O=Something went wrong,CN=WRO...               3.0  3.0  3.0  
C=--,ST=SomeState,L=SomeCity,O=SomeOrganization...               2.0  2.0  2.0  
C=--,ST=SomeState,L=SomeCity,O=SomeOrganization...               2.0  2.0  2.0

<div style="background-image:url(../files/images/slide_background.png);background-repeat: no-repeat;background-size:cover;width:100%;height: 100%;color:white; padding-left: 50px">
<div style="font-weight: bold; font-size: 15em; text-align: center; padding-bottom:30px;">Thanks</div>
<ul style="font-size: 3em;  padding-bottom: 100px">
    <li style="list-style-image: url(../files/images/twitter_icon.png)">@luisdcamachog</li>
    <li style="list-style-image: url(../files/images/square-linkedin-512.png)">https://www.linkedin.com/in/luisdcamachog</li>
    <li style="list-style-image: url(../files/images/github.png)">https://github.com/LuisDavidCamacho/high-performance-pandas</li>
    <li style="list-style-image: url(../files/images/email.png)">luis.camacho@cyxtera.com</li>
</ul>
<div style="widht:150px;heigth: 100px; position: absolute;bottom: 4%; right: 0; width: 300px; height: 100px; background-image:url(../files/images/pycon_title.svg);background-repeat: no-repeat;background-size:contain; padding-right: 8%; padding-botom:10%"></div>    
</div>